<a href="https://colab.research.google.com/github/pikaduck/twt-sentiment-analysis/blob/main/notebooks/twt_sentiment_analysis_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tqdm import tqdm
import pandas as pd
from sklearn.model_selection import train_test_split
import spacy
import spacy_transformers

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
! python -m spacy download en_core_web_trf
! pip install spacy-transformers

In [2]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/test.csv')

df_train0 = df_train.loc[df_train['sentiment'] == 0]
df_train4 = df_train.loc[df_train['sentiment'] == 4]

df_train0 = df_train0.sample(frac = 1).iloc[:5000]
df_train4 = df_train4.sample(frac = 1).iloc[:5000]

df_train = pd.concat([df_train0, df_train4])

df_train['sentiment'] = df_train['sentiment'].replace(4, 1)
df_test = df_test.loc[df_test['sentiment'] != 2]
df_test['sentiment'] = df_test['sentiment'].replace(4, 1)

train, dev = train_test_split(df_train, train_size = 0.85, random_state = 42, shuffle = True)
train.shape, dev.shape

<ipython-input-2-0017da24350f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['sentiment'] = df_test['sentiment'].replace(4, 1)


((8500, 10), (1500, 10))

In [3]:
test = df_test

In [48]:
train['tuple'] = train.apply(lambda x : (x['cleaned_tweet'], x['sentiment']), axis = 1)
train_tuples = train['tuple'].to_list()

dev['tuple'] = dev.apply(lambda x : (x['cleaned_tweet'], x['sentiment']), axis = 1)
dev_tuples = dev['tuple'].to_list()

test['tuple'] = test.apply(lambda x : (x['cleaned_tweet'], x['sentiment']), axis = 1)
test_tuples = test['tuple'].to_list()

In [54]:
nlp = spacy.load('en_core_web_trf')
nlp.pipe_names

['transformer', 'tagger', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']

In [73]:
def convert2doc(data):
  # texts, labels = zip(*data)
  texts = []
  for doc, label in tqdm(nlp.pipe(data, as_tuples = True, batch_size = 1000), total = len(data)):
    # print(doc)
    if label == 0:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1
    else:
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
    texts.append(doc)
  return texts

In [74]:
train_docs = convert2doc(train_tuples)
dev_docs = convert2doc(dev_tuples)
test_docs = convert2doc(test_tuples)

100%|██████████| 359/359 [00:46<00:00,  7.64it/s]


In [81]:
dev_docs[0].cats

{'positive': 1, 'negative': 0}

In [82]:
from spacy.tokens import DocBin

train_doc_bin = DocBin(docs = train_docs)
dev_doc_bin = DocBin(docs = dev_docs)
test_doc_bin = DocBin(docs = test_docs)

train_doc_bin.to_disk('/content/drive/MyDrive/Colab Notebooks/train.spacy')
dev_doc_bin.to_disk('/content/drive/MyDrive/Colab Notebooks/dev.spacy')
test_doc_bin.to_disk('/content/drive/MyDrive/Colab Notebooks/test.spacy')

In [23]:
! python -m spacy init fill-config /content/base_config.cfg /content/config.cfg

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-06 12:45:42.319443: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
✔ Auto-filled config with all values
✔ Saved config
/content/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [83]:
! python -m spacy train config.cfg --verbose --output /content/output

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-06 15:37:23.359094: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
ℹ Saving to output directory: /content/output
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2023-02-06 15:37:25,021] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2023-02-06 15:37:25,033] [DEBUG] Loading corpus from path: /content/drive/MyDrive/Colab Notebooks/dev.spacy
DEBUG:spacy:Loading corpus from path: /content/drive/MyDrive/Colab Notebooks/dev.spacy
[2023-02-06 15:37:25,035] [DEBUG] Loading corpus from path: /content/drive/MyDrive/Colab Notebooks/train.spacy
DEBUG:spacy:Loading corpus from path: /content/drive/MyDrive/Colab Notebooks/train.spacy
[2023-02-06 15:37:25,035] [INFO] Pipeline: ['textcat']
INF

In [85]:
text = "it was such a great film oh my god"

nlp = spacy.load('/content/output/model-best')
demo = nlp(text)

demo.cats

{'positive': 0.643597424030304, 'negative': 0.35640251636505127}

In [86]:
! mv /content/output /content/drive/MyDrive/Colab\ Notebooks/